### Categorical_encoder_generator

In [ ]:
import os
import sys

def add_to_path(new_path: str):
    module_path = os.path.abspath(os.path.join(new_path))
    if module_path not in sys.path:
        sys.path.append(module_path)

is_colab = True

if is_colab:
    !git clone https://github.com/pdkary/Karys.git
    !cd Karys && git fetch && git pull
    !cd Karys && pip install -r requirements.txt --quiet
    add_to_path('Karys/')
    from google.colab import drive
    drive.mount('/content/drive')
    !cd Karys && pip install -r requirements.txt --quiet
else:
    add_to_path('../../')
    !cd ../../ && pip install -r requirements.txt --quiet

In [ ]:
from data.configs.ImageDataConfig import ImageDataConfig
from data.wrappers.ImageDataWrapper import ImageDataWrapper
import numpy as np

if is_colab:
    base_path = "drive/MyDrive/Colab/Seefood/Fruit"
else:
    base_path = "../discriminator/test_input/Fruit"

def map_to_range(input_arr,new_max,new_min):
    img_max = float(np.max(input_arr))
    img_min = float(np.min(input_arr))
    old_range = float(img_max - img_min + 1e-7)
    new_range = (new_max - new_min)
    return new_range*(input_arr - img_min)/old_range + float(new_min)
  
load_scale_function = lambda x: map_to_range(x, 1.0, -1.0)

image_config = ImageDataConfig(image_shape=(128,128,3),image_type=".jpg", preview_rows=4, preview_cols=2, load_n_percent=20, load_scale_func=load_scale_function)

data_wrapper = ImageDataWrapper.load_from_labelled_directories(base_path + '/', image_config, validation_percentage=0.1)
classification_labels = list(set(data_wrapper.image_labels.values()))

##Quick stats about loaded images
print("----------LOADED IMAGE CATEGORIES----------")
print(classification_labels)
print("NUM IMAGES: ",len(data_wrapper.image_labels))

print("----------LOADED IMAGE STATS----------")
value_count_dict = {v:0 for v in set(data_wrapper.image_labels.values())}
for x in data_wrapper.image_labels.values():
    value_count_dict[x]+=1
print(value_count_dict)
print("Max: ",np.max(list(value_count_dict.values())), max(value_count_dict,key=value_count_dict.get))
print("Min: ",np.min(list(value_count_dict.values())), min(value_count_dict,key=value_count_dict.get))
print("Mean: ",np.mean(list(value_count_dict.values())))
print("Median: ",np.median(list(value_count_dict.values())))
print("Deviation: ",np.std(list(value_count_dict.values())))

In [ ]:
LOAD_FROM_SAVED = True

In [ ]:
from data.saved_models.SavedModelService import SavedModelService
from models.EncoderModel import EncoderModel
from models.ClassificationModel import ClassificationModel
from models.GenerationModel import GenerationModel
from models.EncodedClassificationModel import EncodedClassificationModel
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy, Reduction

if LOAD_FROM_SAVED:
  if is_colab:
    model_output_path = "drive/MyDrive/Colab/Seefood/models"
    model_reference_path = "drive/MyDrive/Colab/Seefood/models/model_ref.json"
  else:
    model_output_path = "./test_model_output"
    model_reference_path = "../encoder/test_model_output/model_ref.json"

  saved_model_service = SavedModelService(model_reference_path)

  encoder_optimizer = Adam(learning_rate=4e-6)#, decay=1e-9)
  encoder_loss = CategoricalCrossentropy(reduction=Reduction.SUM)

  vectorizer_optimizer = Adam(learning_rate=4e-6)#, decay=1e-9)
  vectorizer_loss = CategoricalCrossentropy(reduction=Reduction.SUM)

  classifier_optimizer = Adam(learning_rate=4e-6)#, decay=1e-9)
  classifier_loss = CategoricalCrossentropy(reduction=Reduction.SUM)

  generator_optimizer = Adam(learning_rate=4e-6)#, decay=1e-9)
  generator_loss = CategoricalCrossentropy(reduction=Reduction.SUM)

  encoder: EncoderModel = saved_model_service.load_model_by_name("FruitEncoder", EncoderModel, encoder_optimizer, encoder_loss)
  label_vectorizer: EncoderModel = saved_model_service.load_model_by_name("FruitLabelVectorizer", EncoderModel, vectorizer_optimizer, vectorizer_loss)
  classifier: ClassificationModel = saved_model_service.load_model_by_name("EncodedFruitClassifier", ClassificationModel, classification_labels, ["generated"], classifier_optimizer, classifier_loss)
  generator: GenerationModel = saved_model_service.load_model_by_name("FruitGenerator", GenerationModel, optimizer=generator_optimizer, loss=generator_loss)


In [ ]:
from models.ClassificationModel import ClassificationModel
from models.EncoderModel import EncoderModel

from tensorflow.keras.layers import Conv2D, Dense, LeakyReLU, MaxPooling2D, Flatten, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy, Reduction

if not LOAD_FROM_SAVED:
  ENCODED_DIM = 512
  a = 0.08

  encoder_layers = [
      Conv2D(64,3), BatchNormalization(), LeakyReLU(a),
      Conv2D(64,3), BatchNormalization(), LeakyReLU(a),
      MaxPooling2D(),
      Conv2D(128,3), BatchNormalization(), LeakyReLU(a),
      Conv2D(128,3), BatchNormalization(), LeakyReLU(a),
      MaxPooling2D(),
      Conv2D(256,3), BatchNormalization(), LeakyReLU(a),
      Conv2D(256,3), BatchNormalization(), LeakyReLU(a),
      Conv2D(256,3), BatchNormalization(), LeakyReLU(a),
      MaxPooling2D(),
      Conv2D(512,3), BatchNormalization(), LeakyReLU(a),
      Conv2D(512,3), BatchNormalization(), LeakyReLU(a),
      Conv2D(512,3), BatchNormalization(), LeakyReLU(a),
      MaxPooling2D(),
      Flatten(),
      Dense(ENCODED_DIM), Activation('tanh'),
      Dense(ENCODED_DIM), Activation('sigmoid'),
  ]

  encoder_optimizer = Adam(learning_rate=4e-6)#, decay=1e-9)
  encoder_loss = CategoricalCrossentropy(reduction=Reduction.SUM)
  encoder = EncoderModel(image_config.image_shape, ENCODED_DIM, encoder_layers, encoder_optimizer, encoder_loss)


In [ ]:
from models.ClassificationModel import ClassificationModel
from models.EncoderModel import EncoderModel

from tensorflow.keras.layers import Conv2D, Dense, LeakyReLU, MaxPooling2D, Flatten, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy, Reduction

if not LOAD_FROM_SAVED:
      vectorizer_layers = [
            Dense(256), Activation('tanh'),
            Dense(256), Activation('tanh'),
            Dense(256), Activation('tanh'),
            Dense(len(classification_labels)+1), Activation('softmax'),
      ]

      vectorizer_optimizer = Adam(learning_rate=4e-6)#, decay=1e-9)
      vectorizer_loss = CategoricalCrossentropy(reduction=Reduction.SUM)
      label_vectorizer = EncoderModel([len(classifier)+1], ENCODED_DIM, vectorizer_layers, vectorizer_optimizer, vectorizer_loss)

In [ ]:
from models.ClassificationModel import ClassificationModel
from models.EncoderModel import EncoderModel

from tensorflow.keras.layers import Conv2D, Dense, LeakyReLU, MaxPooling2D, Flatten, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy, Reduction

if not LOAD_FROM_SAVED:
    classifier_layers = [
        Dense(4096), Activation('tanh'),
        Dense(4096), Activation('tanh'),
        Dense(len(classification_labels)+1), Activation('softmax'),
    ]

    classifier_optimizer = Adam(learning_rate=4e-6)#, decay=1e-9)
    classifier_loss = CategoricalCrossentropy(reduction=Reduction.SUM)
    classifier = ClassificationModel([ENCODED_DIM],classification_labels, ["generated"], classifier_layers, classifier_optimizer, classifier_loss)

In [ ]:
from models.GenerationModel import GenerationModel
from tensorflow.keras.layers import Conv2D, Dense, LeakyReLU, Activation, Flatten, Reshape, UpSampling2D, BatchNormalization, GaussianNoise
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy, Reduction
from tensorflow_addons.layers import InstanceNormalization

if not LOAD_FROM_SAVED:
  ENCODED_DIM = encoder.output_shape[-1]
  a = 0.08
  stddev = 0.05

  gen_layers = [
      Dense(512), Activation('tanh'),
      Dense(512), Activation('tanh'),
      Dense(512), Activation('tanh'),    
      Dense(4096), Activation('tanh'),
      Reshape((2,2,1024)),
      UpSampling2D(),#GaussianNoise(stddev),
      Conv2D(512,3,padding="same"),BatchNormalization(), LeakyReLU(a),
      Conv2D(512,3,padding="same"),BatchNormalization(), LeakyReLU(a),
      Conv2D(512,3,padding="same"),BatchNormalization(), LeakyReLU(a),
      UpSampling2D(),#GaussianNoise(stddev),
      Conv2D(256,3,padding="same"), BatchNormalization(), LeakyReLU(a),
      Conv2D(256,3,padding="same"), BatchNormalization(), LeakyReLU(a),
      Conv2D(256,3,padding="same"), BatchNormalization(), LeakyReLU(a),
      UpSampling2D(),#GaussianNoise(stddev),
      Conv2D(128,3,padding="same"), BatchNormalization(), LeakyReLU(a),
      Conv2D(128,3,padding="same"), BatchNormalization(), LeakyReLU(a),
      Conv2D(128,3,padding="same"), BatchNormalization(), LeakyReLU(a),
      UpSampling2D(),#GaussianNoise(stddev),
      Conv2D(64,3,padding="same"), BatchNormalization(), LeakyReLU(a),
      Conv2D(64,3,padding="same"), BatchNormalization(), LeakyReLU(a),
      Conv2D(64,3,padding="same"), BatchNormalization(), LeakyReLU(a),
      UpSampling2D(),#GaussianNoise(stddev),
      Conv2D(32,3,padding="same"), BatchNormalization(), LeakyReLU(a),
      Conv2D(32,3,padding="same"), BatchNormalization(), LeakyReLU(a),
      Conv2D(32,3,padding="same"), BatchNormalization(), LeakyReLU(a),
      UpSampling2D(),#GaussianNoise(stddev),
      Conv2D(16,3,padding="same"),BatchNormalization(),  LeakyReLU(a),
      Conv2D(16,3,padding="same"),BatchNormalization(),  LeakyReLU(a),
      Conv2D(3,3,padding="same"), BatchNormalization(), Activation('tanh'),
  ]

  optimizer = Adam(learning_rate=4e-6)#, decay=1e-9)
  loss = CategoricalCrossentropy(reduction=Reduction.SUM)
  generator = GenerationModel(ENCODED_DIM, image_config.image_shape,gen_layers,optimizer,loss)

In [ ]:
from models.CategoricalEncoderGenerator import CategoricalEncoderGenerator
categorical_encoder_generator = CategoricalEncoderGenerator(encoder, label_vectorizer, generator, classifier)
categorical_encoder_generator.build()

In [ ]:
from plotting.TrainPlotter import TrainPlotter
from trainers.CategoricalEncoderGeneratorTrainer import CategoricalEncoderGeneratorTrainer

epochs=3250
trains_per_test=250

train_columns = ["Encoder Loss", "Label Vectorizer loss", "Decoder Loss", "Classifier Loss", "Test Encoder Loss", "Test Label Vectorizer loss", "Test Decoder Loss", "Test Classifier Loss"]
loss_plot = TrainPlotter(moving_average_size=250, labels=train_columns)
trainer = CategoricalEncoderGeneratorTrainer(categorical_encoder_generator, data_wrapper,0.0, 1.0, 0.0, 1.0)

if is_colab:
  image_output_path = "drive/MyDrive/Colab/Seefood/output"
else:
  image_output_path = "./test_output"

e_test_loss, v_test_loss, d_test_loss, c_test_loss = 0, 0, 0
for i in range(epochs):
  loss_plot.start_epoch()  
  e_loss, v_loss, d_loss, c_loss = trainer.train(16, 1)

  if i % trains_per_test == 0 and i != 0:
    e_test_loss, v_test_loss, d_test_loss, c_test_loss = trainer.test(16,1)
    gen_filename = image_output_path + "/generated-" + str(i) + ".jpg"
    lbl_gen_filename = image_output_path + "/generated-" + str(i) + ".jpg"

    encoding_real_filename = image_output_path + "/encoded-real-" + str(i) + ".jpg"
    encoding_label_filename = image_output_path + "/encoded-label-" + str(i) + ".jpg"
    encoding_gen_filename = image_output_path + "/encoded-gen-" + str(i) + ".jpg"
    encoding_label_gen_filename = image_output_path + "/encoded-label_gen-" + str(i) + ".jpg"

    classification_real_filename = image_output_path + "/classified-real-" + str(i) + ".jpg"
    classification_label_filename = image_output_path + "/classified-label-" + str(i) + ".jpg"
    classification_gen_filename = image_output_path + "/classified-gen-" + str(i) + ".jpg"
    classification_label_gen_filename = image_output_path + "/classified-label_gen-" + str(i) + ".jpg"

    data_wrapper.save_generated_images(gen_filename, trainer.most_recent_generated)
    data_wrapper.save_generated_images(lbl_gen_filename, trainer.most_recent_label_generated)

    data_wrapper.save_encoded_images(encoding_real_filename, trainer.most_recent_real_encoding, img_size=32)
    data_wrapper.save_encoded_images(classification_label_filename, trainer.most_recent_label_encodings, img_size=32)
    data_wrapper.save_encoded_images(encoding_gen_filename, trainer.most_recent_gen_encoding, img_size=32)
    data_wrapper.save_encoded_images(encoding_label_gen_filename, trainer.most_recent_label_gen_encoding, img_size=32)


    data_wrapper.save_classified_images(classification_real_filename, trainer.most_recent_real_classifications, img_size=32)
    data_wrapper.save_classified_images(classification_label_filename, trainer.most_recent_label_classifications, img_size=32)
    data_wrapper.save_classified_images(classification_gen_filename, trainer.most_recent_gen_classifications, img_size=32)
    data_wrapper.save_classified_images(classification_label_gen_filename, trainer.most_recent_label_gen_classifications, img_size=32)
      
  loss_plot.batch_update([e_loss, v_loss, d_loss, c_loss, e_test_loss, v_test_loss, d_test_loss, c_test_loss])
  loss_plot.log_epoch()